In [1]:
%%time
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd
import os
from itertools import chain
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime, timedelta

Wall time: 26.6 s


## Write to database function

In [2]:
def write_to(dataframe,table_name):

    db_params = {
        'host': 'ec2-34-251-233-253.eu-west-1.compute.amazonaws.com',
        'database': 'd6smqg5adrljks',
        'user': 'tzjbmiqqkxkuxu',
        'password': '1630e53a904f24ef0d85affbd499bd6f3cbbc1c43c47ca8d4180ba8841ee7676',
    }

    connection = psycopg2.connect(**db_params)

    # Create a SQLAlchemy engine
    engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

    dataframe.to_sql(table_name, engine, if_exists='append', index=False)

    connection.commit()
    connection.close()
    print('Push to database successfully')
    

    
def read_from(table):
    
    db_params = {
        'host': 'ec2-34-251-233-253.eu-west-1.compute.amazonaws.com',
        'database': 'd6smqg5adrljks',
        'user': 'tzjbmiqqkxkuxu',
        'password': '1630e53a904f24ef0d85affbd499bd6f3cbbc1c43c47ca8d4180ba8841ee7676',
        }

    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()

    query = f"SELECT * FROM {table}"
    cursor.execute(query)

    result = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    cursor.close()
    connection.close()

    # Create a DataFrame from the fetched data
    from_db = pd.DataFrame(result, columns=column_names)
    return from_db
    




# Leagues and matches, code above has been added to code below 


## Working code block updated to selenium 4

In [6]:
%%time
service = Service(executable_path=r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe')
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")


driver = webdriver.Chrome(service=service,options=options)


driver.get('https://www.sofascore.com/tournament/football/spain/laliga/8#52376')
WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//div[@class='sc-fqkvVR clkedU']/div/a")))
driver.execute_script("window.stop();")
matches =driver.find_elements(By.XPATH,"//div[@class='sc-fqkvVR clkedU']/div/a")
match = [con.get_attribute('innerText') for con in matches]
print('Level 1: League level')
match_links =driver.find_elements(By.XPATH,"//div[@class='sc-fqkvVR clkedU']/div/a[@href]")
match_link = [con.get_attribute('href') for con in match_links] 
print(match)
elements = driver.find_elements(By.CLASS_NAME, 'sc-aXZVg.dIaSSb')
elements[1].click()
time.sleep(2)
matches2 =driver.find_elements(By.XPATH,"//div[@class='sc-fqkvVR clkedU']/div/a")
match2 = [con.get_attribute('innerText') for con in matches2]
print('Level 1: League level next page')
match_links2 =driver.find_elements(By.XPATH,"//div[@class='sc-fqkvVR clkedU']/div/a[@href]")
match_link2 = [con.get_attribute('href') for con in match_links2]
driver.quit()
print(match2)
match = match + match2
match_link = match_link + match_link2
match  = [item.replace('\n', ',').split(',') for item in match]
print(match)
data = [sublist[:4] for sublist in match]
df = pd.DataFrame(data, columns=['Date', 'Time', 'Home_Team', 'Away_Team'])
df['match_links'] = match_link
df['Date'] = pd.to_datetime(df['Date'])
today = datetime.today().date()
yesterday = (datetime.today() + timedelta(days=-1)).date()
df = df[df['Date'].dt.date == today]
df

def matches_get(url):
    service = Service(executable_path=r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--blink-settings=imagesEnabled=false')
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")


    driver = webdriver.Chrome(service=service,options=options)

    driver.get(url)
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//div[@class='sc-jlZhew kgghii']/div/div")))
    driver.execute_script("window.stop();")
    team_links =driver.find_elements(By.XPATH,"//div[@class='sc-fqkvVR gWLOaC']/div/a[@href]")
    team_link = [con.get_attribute('href') for con in team_links]
    print(team_link)
    print('Level 2: Match level')
    team_titles = driver.find_elements(By.CLASS_NAME,"sc-fqkvVR.gWLOaC")
    team_title = [con.get_attribute('innerText') for con in team_titles]
    print(team_title)


    driver.quit()
    

    def ind_team_stats(url):
        service = Service(executable_path=r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe')
        options = Options()
        ua = UserAgent()
        userAgent = ua.random
        options.add_argument(f'user-agent={userAgent}')
        options.add_argument('--blink-settings=imagesEnabled=false')
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--window-size=1920,1080")


        driver = webdriver.Chrome(service = service,options=options)

        driver.get(url)
        WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//div[@class='sc-fqkvVR Ksgce']/div")))
        driver.execute_script("window.stop();")
        team_ages =driver.find_elements(By.CLASS_NAME,"sc-fqkvVR.sc-dcJsrY.jqdASS.ijIoLB")
        team_age = [con.get_attribute('innerText') for con in team_ages]
        team_age = team_age[1]
        team_age = [team_age.replace('\n', ',')]
        values = team_age[0].split(',')
        df_team_age = pd.DataFrame([values])
        df_team_age =df_team_age.drop([1, 3,5,7], axis=1)
        df_team_age.rename(columns = {0:'Total Players', 2:'Average player age',
                                      4:'Foreign players',6:'National team players'}, inplace = True)

        print('Level 3: Individual matches level')
        team_summarys =driver.find_elements(By.CLASS_NAME,"sc-fqkvVR.Ksgce")
        team_summary = [con.get_attribute('innerText') for con in team_summarys]
        driver.quit()
        df_team_summary = pd.DataFrame()
        # Loop through the list and split each element by '\n'
        for item in team_summary:
            values = item.split('\n')
            headers = values[::2]
            values = values[1::2]
            # Create a temporary DataFrame from the current item
            temp_df = pd.DataFrame([values], columns=headers)
            # Append the temporary DataFrame to the main DataFrame
            df_team_summary = pd.concat([df_team_summary, temp_df], axis=1)
        df_team_summary = df_team_summary.transpose()
        df_team_summary.index.name = 'Metric'
        df_team_summary.rename(columns = {0:'Values'}, inplace = True)
        df_team_summary = df_team_summary.transpose()
        

        print(df_team_age)
        return df_team_summary
    

    home_url = team_link[0]
    away_url = team_link[1]

    # Fetch data for the home team
    home_df = ind_team_stats(home_url)
    home_df.reset_index(drop=True, inplace=True)
    home_df.columns = ['Home_' + col for col in home_df.columns]

    away_df = ind_team_stats(away_url)
    away_df.reset_index(drop=True, inplace=True)
    away_df.columns = ['Away_' + col for col in away_df.columns]

    # Concatenate the home and away DataFrames horizontally
    teams_df = pd.concat([home_df, away_df], axis=1)
    teams_df.columns =teams_df.columns.str.replace(' ', '_')
    return teams_df

combined_df = []

for url in df.match_links:
    data_df = matches_get(url)
    
    combined_df.append(data_df)
    
combined = pd.concat(combined_df,join = 'outer',ignore_index=True)

move_to_database = pd.concat([df.reset_index(drop=True), combined], axis=1,ignore_index=False)
#calls the function that gets writes data to the dataase
write_to(move_to_database,'matches')


Level 1: League level
['24/09/23\nFT\nAtl. Madrid\nReal Madrid\n3\n3\n1\n1', '26/09/23\nFT\nSevilla\nAlmería\n5\n5\n1\n1', '26/09/23\nFT\nMallorca\nBarcelona\n2\n2\n2\n2', '27/09/23\nFT\nAthletic Club\nGetafe\n2\n2\n2\n2', '27/09/23\nFT\nVillarreal\nGirona\n1\n1\n2\n2', '27/09/23\nFT\nReal Madrid\nLas Palmas\n2\n2\n0\n0', '27/09/23\nFT\nCádiz\nRayo Vallecano\n0\n0\n0\n0', '27/09/23\nFT\nValencia\nReal Sociedad\n0\n0\n1\n1', '18:00\n-\nGranada\nBetis', '18:00\n-\nCelta\nAlavés']
Level 1: League level next page
['20:30\n-\nOsasuna\nAtl. Madrid', '29/09/23\n20:00\nBarcelona\nSevilla', '30/09/23\n13:00\nGetafe\nVillarreal', '30/09/23\n15:15\nRayo Vallecano\nMallorca', '30/09/23\n17:30\nGirona\nReal Madrid', '30/09/23\n20:00\nReal Sociedad\nAthletic Club', '01/10/23\n13:00\nAlmería\nGranada', '01/10/23\n15:15\nAlavés\nOsasuna', '01/10/23\n20:00\nAtl. Madrid\nCádiz', '01/10/23\n20:00\nBetis\nValencia']
[['24/09/23', 'FT', 'Atl. Madrid', 'Real Madrid', '3', '3', '1', '1'], ['26/09/23', 'FT', 

In [8]:
move_to_database


,Date,Time,Home_Team,Away_Team,match_links,Home_Matches,Home_Goals_scored,Home_Goals_conceded,Home_Assists,Home_Goals_scored,...,Away_Duels_won,Away_Ground_duels_won,Away_Aerial_duels_won,Away_Possession_lost,Away_throwIns,Away_goalKicks,Away_Offsides,Away_Fouls,Away_Yellow,Away_Red_cards
0,2023-09-28 18:00:00,-,Granada,Betis,https://www.sofascore.com/granada-real-betis/q...,6,9,17,7,1.5,...,51 (48%),42.2 (50.1%),8.8 (40.2%),120.2,14.7,7.2,2,13.3,2.3,0
1,2023-09-28 18:00:00,-,Celta,Alavés,https://www.sofascore.com/deportivo-alaves-cel...,6,6,10,4,1,...,54.7 (50.5%),35.2 (49.1%),19.5 (53.4%),151.2,26.2,10.2,2.2,14.8,2.3,0
2,2023-09-28 20:30:00,-,Osasuna,Atl. Madrid,https://www.sofascore.com/atletico-madrid-osas...,6,7,8,3,1.2,...,51.6 (54%),40 (52.5%),11.6 (59.8%),126.6,19,7.6,1.4,14,2.6,0


# get the new dataframe with scores: 

## chosen approach; get them directly from the premier league link.

In [9]:
%%time
service = Service(executable_path=r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe')
options = Options()
ua = UserAgent()
userAgent = ua.random
options.add_argument(f'user-agent={userAgent}')
options.add_argument('--blink-settings=imagesEnabled=false')
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")


driver = webdriver.Chrome(service=service,options=options)


driver.get('https://www.sofascore.com/tournament/football/spain/laliga/8#52376')
WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//div[@class='sc-fqkvVR clkedU']/div/a")))
driver.execute_script("window.stop();")
matches =driver.find_elements(By.XPATH,"//div[@class='sc-fqkvVR clkedU']/div/a")
match = [con.get_attribute('innerText') for con in matches]
print('Level 1: League level')
#match_links =driver.find_elements_by_xpath("//div[@class='sc-fqkvVR clkedU']/div/a[@href]")
#match_link = [con.get_attribute('href') for con in match_links] 
print(match)
elements = driver.find_elements(By.CLASS_NAME, 'sc-aXZVg.dIaSSb')
elements[0].click()
time.sleep(2)
matches2 =driver.find_elements(By.XPATH,"//div[@class='sc-fqkvVR clkedU']/div/a")
match2 = [con.get_attribute('innerText') for con in matches2]
print('Level 1: League level next page')
#match_links2 =driver.find_elements_by_xpath("//div[@class='sc-fqkvVR clkedU']/div/a[@href]")
#match_link2 = [con.get_attribute('href') for con in match_links2]
driver.quit()
print(match2)
match = match + match2
#match_link = match_link + match_link2
match  = [item.replace('\n', ',').split(',') for item in match]
clean = [inner_list[:4] + inner_list[5:7] if len(inner_list) == 8 else inner_list for inner_list in match]
data = [sublist[:6] for sublist in clean]
my_list = [inner_list for inner_list in data if 'FT' in inner_list]
score_df = pd.DataFrame(my_list, columns=['Date', 'Time', 'Home_Team', 'Away_Team','Home_score','Away_score'])
score_df['Date'] = pd.to_datetime(score_df['Date'])
today = datetime.today().date()
yesterday = (datetime.today() + timedelta(days=-1)).date()
score_df = score_df[score_df['Date'].dt.date == yesterday]
write_to(score_df,'match_outcome')

Level 1: League level
['24/09/23\nFT\nAtl. Madrid\nReal Madrid\n3\n3\n1\n1', '26/09/23\nFT\nSevilla\nAlmería\n5\n5\n1\n1', '26/09/23\nFT\nMallorca\nBarcelona\n2\n2\n2\n2', '27/09/23\nFT\nAthletic Club\nGetafe\n2\n2\n2\n2', '27/09/23\nFT\nVillarreal\nGirona\n1\n1\n2\n2', '27/09/23\nFT\nReal Madrid\nLas Palmas\n2\n2\n0\n0', '27/09/23\nFT\nCádiz\nRayo Vallecano\n0\n0\n0\n0', '27/09/23\nFT\nValencia\nReal Sociedad\n0\n0\n1\n1', '18:00\n-\nGranada\nBetis', '18:00\n-\nCelta\nAlavés']
Level 1: League level next page
['18/09/23\nFT\nGranada\nGirona\n2\n4', '22/09/23\nFT\nAlavés\nAthletic Club\n0\n2', '23/09/23\nFT\nGirona\nMallorca\n5\n3', '23/09/23\nFT\nOsasuna\nSevilla\n0\n0', '23/09/23\nFT\nBarcelona\nCelta\n3\n2', '23/09/23\nFT\nAlmería\nValencia\n2\n2', '24/09/23\nFT\nReal Sociedad\nGetafe\n4\n3', '24/09/23\nFT\nRayo Vallecano\nVillarreal\n1\n1', '24/09/23\nFT\nLas Palmas\nGranada\n1\n0', '24/09/23\nFT\nBetis\nCádiz\n1\n1']
Push to database successfully
Wall time: 39.4 s


In [10]:
score_df

,Date,Time,Home_Team,Away_Team,Home_score,Away_score
3,2023-09-27,FT,Athletic Club,Getafe,2,2
4,2023-09-27,FT,Villarreal,Girona,1,2
5,2023-09-27,FT,Real Madrid,Las Palmas,2,0
6,2023-09-27,FT,Cádiz,Rayo Vallecano,0,0
7,2023-09-27,FT,Valencia,Real Sociedad,0,1


## method 2 not chosen: get them from each link in the database

In [9]:
import psycopg2
import pandas as pd

db_params = {
    'host': 'ec2-34-251-233-253.eu-west-1.compute.amazonaws.com',
    'database': 'd6smqg5adrljks',
    'user': 'tzjbmiqqkxkuxu',
    'password': '1630e53a904f24ef0d85affbd499bd6f3cbbc1c43c47ca8d4180ba8841ee7676',
}

connection = psycopg2.connect(**db_params)
cursor = connection.cursor()

query = "SELECT * FROM matches"
cursor.execute(query)

result = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
cursor.close()
connection.close()

# Create a DataFrame from the fetched data
from_db = pd.DataFrame(result, columns=column_names)

scores = []
def get_scores(url):
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument('--blink-settings=imagesEnabled=false')
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")
    capa = DesiredCapabilities.CHROME
    capa["pageLoadStrategy"] = "none"

    driver = webdriver.Chrome(r'C:\Users\HP\Downloads\Premier_League_Stats\Loading_with_selenium\chromedriver.exe',options=options,desired_capabilities=capa)


    driver.get(url)
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//span[@class='sc-jEACwC cgJaFs']")))
    driver.execute_script("window.stop();")
    outcomes =driver.find_elements(By.CLASS_NAME, 'sc-fqkvVR.a-DkxQ')
    outcome = [con.get_attribute('innerText') for con in outcomes]
    data  = [item.replace('\n', ',') for item in outcome]
    print('gotten')
    scores.append(data)
for i in from_db.match_links:
    get_scores(i)
    
df = pd.DataFrame([item[0].split(',') for item in scores], columns=['home_Team', 'Score', 'Status', 'away_team'])

gotten
gotten
gotten
gotten
gotten


In [24]:
df 

,Date,Time,Home_Team,Away_Team,Home_score,Away_score
3,2023-09-24,FT,Brighton,Bournemouth,3,1
4,2023-09-24,FT,Chelsea,Aston Villa,0,1
5,2023-09-24,FT,Arsenal,Tottenham,2,2
6,2023-09-24,FT,Liverpool,West Ham,3,1
7,2023-09-24,FT,Sheffield Utd,Newcastle,0,8


In [25]:
from_db

,Date,Time,Home_Team,Away_Team,match_links,Home_Matches,Home_Goals_scored,Home_Goals_conceded,Home_Assists,Home_Penalty_goals,...,Away_Duels_won,Away_Ground_duels_won,Away_Aerial_duels_won,Away_Possession_lost,Away_throwIns,Away_goalKicks,Away_Offsides,Away_Fouls,Away_Yellow,Away_Red_cards
0,2023-09-24,FT,Brighton,Bournemouth,https://www.sofascore.com/bournemouth-brighton...,6,3,1.3,14,1/1,...,52.7 (46.7%),42.5 (49.1%),10.2 (38.9%),137,19.7,8.8,1.5,14.2,1.8,0
1,2023-09-24,FT,Chelsea,Aston Villa,https://www.sofascore.com/aston-villa-chelsea/...,6,0.8,1,3,0/1,...,44.2 (50.4%),36.8 (51.3%),7.3 (46.3%),116.2,17.2,6.7,0.8,11.2,2.7,0
2,2023-09-24,FT,Arsenal,Tottenham,https://www.sofascore.com/arsenal-tottenham-ho...,6,1.8,1,7,3/3,...,53.7 (49.3%),44.3 (51.6%),9.3 (40.9%),122.5,17,7,1.2,12.3,3.5,0
3,2023-09-24,FT,Liverpool,West Ham,https://www.sofascore.com/liverpool-west-ham-u...,6,2.5,0.8,8,1/2,...,53.2 (50.7%),37.7 (52.3%),15.5 (47.2%),124,16.2,10.8,2.5,11.3,2.7,1
4,2023-09-24,FT,Sheffield Utd,Newcastle,https://www.sofascore.com/newcastle-united-she...,6,0.8,2.8,2,0,...,47.8 (50%),38 (49.9%),9.8 (50.4%),120.3,16,7.5,1.2,12,2.8,0
